In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [7]:
# Load the data
file_path = Path('population_pct_within_qrt_mile_alcohol.csv')


In [13]:
df = pd.read_csv(file_path)
df

,Indicator_ID,ind_definition,reportyear,race_eth_code,race_eth_name,geotype,geotypevalue,geoname,county_name,county_fips_id,region_name,region_code,license_type,num_people_qrt,tot_people,pct_of_total
0,774,Percent of Population within 1/4 Mile of Alcoh...,2014,8,Other,CT,6001442100,4421,Alameda,6001,NaN,NaN,Total_licenses,0.0,0.0,NaN
1,774,Percent of Population within 1/4 Mile of Alcoh...,2014,1,AIAN,CD,600190020,Alameda,Alameda,6001,Bay Area,1.0,Total_licenses,159.0,247.0,64.37
2,774,Percent of Population within 1/4 Mile of Alcoh...,2014,3,AfricanAm,CD,600190020,Alameda,Alameda,6001,Bay Area,1.0,Total_licenses,2674.0,4516.0,59.21
3,774,Percent of Population within 1/4 Mile of Alcoh...,2014,2,Asian,CD,600190020,Alameda,Alameda,6001,Bay Area,1.0,Total_licenses,14243.0,22822.0,62.41
4,774,Percent of Population within 1/4 Mile of Alcoh...,2014,4,Latino,CD,600190020,Alameda,Alameda,6001,Bay Area,1.0,Total_licenses,5689.0,8092.0,70.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90445,774,Percent of Population within 1/4 Mile of Alcoh...,2014,7,Multiple,RE,14,Southern California,NaN,0,Southern California,14.0,Total_licenses,181987.0,378200.0,48.12
90446,774,Percent of Population within 1/4 Mile of Alcoh...,2014,5,NHOPI,RE,14,Southern California,NaN,0,Southern California,14.0,Total_licenses,25455.0,43955.0,57.91
90447,774,Percent of Population within 1/4 Mile of Alcoh...,2014,8,Other,RE,14,Southern California,NaN,0,Southern California,14.0,Total_licenses,22985.0,40257.0,57.10
90448,774,Percent of Population within 1/4 Mile of Alcoh...,2014,9,Total,RE,14,Southern California,NaN,0,Southern California,14.0,Total_licenses,9744365.0,18051534.0,53.98


In [14]:
df.dtypes

Indicator_ID        int64
ind_definition     object
reportyear          int64
race_eth_code       int64
race_eth_name      object
geotype            object
geotypevalue        int64
geoname            object
county_name        object
county_fips_id      int64
region_name        object
region_code       float64
license_type       object
num_people_qrt    float64
tot_people        float64
pct_of_total      float64
dtype: object

In [15]:
# Define features data
df_dropna = df.copy().dropna()
df_new = df_dropna.drop(columns=["Indicator_ID", "ind_definition", "geotype","reportyear", "geotypevalue", "geoname", "county_fips_id"])

In [16]:
df_new

,race_eth_code,race_eth_name,county_name,region_name,region_code,license_type,num_people_qrt,tot_people,pct_of_total
1,1,AIAN,Alameda,Bay Area,1.0,Total_licenses,159.0,247.0,64.37
2,3,AfricanAm,Alameda,Bay Area,1.0,Total_licenses,2674.0,4516.0,59.21
3,2,Asian,Alameda,Bay Area,1.0,Total_licenses,14243.0,22822.0,62.41
4,4,Latino,Alameda,Bay Area,1.0,Total_licenses,5689.0,8092.0,70.30
5,7,Multiple,Alameda,Bay Area,1.0,Total_licenses,2490.0,4047.0,61.53
...,...,...,...,...,...,...,...,...,...
90310,7,Multiple,Yuba,Sacramento Area,8.0,Total_licenses,71.0,153.0,46.41
90311,5,NHOPI,Yuba,Sacramento Area,8.0,Total_licenses,0.0,4.0,0.00
90312,8,Other,Yuba,Sacramento Area,8.0,Total_licenses,0.0,7.0,0.00
90313,9,Total,Yuba,Sacramento Area,8.0,Total_licenses,1583.0,3456.0,45.80


In [20]:
#Create a list of condition to turn pct_of_total to categorical variables
pop_density = [
    (df_new['pct_of_total'] <= 50.00),
    (df_new['pct_of_total'] > 50.00)
    ]
values = ['low_density', 'high_density']

df_new['density'] = np.select(pop_density, values)


In [21]:
df_new

,race_eth_code,race_eth_name,county_name,region_name,region_code,license_type,num_people_qrt,tot_people,pct_of_total,density
1,1,AIAN,Alameda,Bay Area,1.0,Total_licenses,159.0,247.0,64.37,high_density
2,3,AfricanAm,Alameda,Bay Area,1.0,Total_licenses,2674.0,4516.0,59.21,high_density
3,2,Asian,Alameda,Bay Area,1.0,Total_licenses,14243.0,22822.0,62.41,high_density
4,4,Latino,Alameda,Bay Area,1.0,Total_licenses,5689.0,8092.0,70.30,high_density
5,7,Multiple,Alameda,Bay Area,1.0,Total_licenses,2490.0,4047.0,61.53,high_density
...,...,...,...,...,...,...,...,...,...,...
90310,7,Multiple,Yuba,Sacramento Area,8.0,Total_licenses,71.0,153.0,46.41,low_density
90311,5,NHOPI,Yuba,Sacramento Area,8.0,Total_licenses,0.0,4.0,0.00,low_density
90312,8,Other,Yuba,Sacramento Area,8.0,Total_licenses,0.0,7.0,0.00,low_density
90313,9,Total,Yuba,Sacramento Area,8.0,Total_licenses,1583.0,3456.0,45.80,low_density


In [27]:
df_density = df_new.drop(columns=["pct_of_total"])

In [28]:
df_density

,race_eth_code,race_eth_name,county_name,region_name,region_code,license_type,num_people_qrt,tot_people,density
1,1,AIAN,Alameda,Bay Area,1.0,Total_licenses,159.0,247.0,high_density
2,3,AfricanAm,Alameda,Bay Area,1.0,Total_licenses,2674.0,4516.0,high_density
3,2,Asian,Alameda,Bay Area,1.0,Total_licenses,14243.0,22822.0,high_density
4,4,Latino,Alameda,Bay Area,1.0,Total_licenses,5689.0,8092.0,high_density
5,7,Multiple,Alameda,Bay Area,1.0,Total_licenses,2490.0,4047.0,high_density
...,...,...,...,...,...,...,...,...,...
90310,7,Multiple,Yuba,Sacramento Area,8.0,Total_licenses,71.0,153.0,low_density
90311,5,NHOPI,Yuba,Sacramento Area,8.0,Total_licenses,0.0,4.0,low_density
90312,8,Other,Yuba,Sacramento Area,8.0,Total_licenses,0.0,7.0,low_density
90313,9,Total,Yuba,Sacramento Area,8.0,Total_licenses,1583.0,3456.0,low_density


In [29]:
#Split the Data into Training and Testing
# Create our features
y = pd.DataFrame(df_density['density'])
X = pd.get_dummies(df_density.drop(columns="density"))

In [30]:
X.describe()

,race_eth_code,region_code,num_people_qrt,tot_people,race_eth_name_AIAN,race_eth_name_AfricanAm,race_eth_name_Asian,race_eth_name_Latino,race_eth_name_Multiple,race_eth_name_NHOPI,...,region_name_Northeast Sierra,region_name_Northern Sacramento Valley,region_name_Sacramento Area,region_name_San Diego,region_name_San Joaquin Valley,region_name_San Luis Obispo,region_name_Santa Barbara,region_name_Shasta,region_name_Southern California,license_type_Total_licenses
count,14621.000000,14621.000000,1.462100e+04,1.462100e+04,14621.000000,14621.000000,14621.000000,14621.000000,14621.000000,14621.000000,...,14621.000000,14621.000000,14621.000000,14621.000000,14621.000000,14621.000000,14621.000000,14621.000000,14621.000000,14621.0
mean,4.991519,8.164968,7.445648e+03,1.497877e+04,0.113057,0.109158,0.112851,0.117707,0.116340,0.096300,...,0.051228,0.019629,0.081116,0.038985,0.181246,0.016278,0.017235,0.011832,0.252240,1.0
std,2.595731,4.635320,8.159632e+04,1.305161e+05,0.316673,0.311848,0.316422,0.322272,0.320643,0.295012,...,0.220469,0.138727,0.273023,0.193566,0.385235,0.126547,0.130152,0.108135,0.434313,0.0
min,1.000000,1.000000,0.000000e+00,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,3.000000,4.000000,6.000000e+00,3.000000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,5.000000,9.000000,6.800000e+01,2.500000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,7.000000,14.000000,9.070000e+02,3.006000e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.0
max,9.000000,14.000000,6.519275e+06,9.818605e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


In [31]:
# Check the balance of our target values
y['density'].value_counts()

low_density     10480
high_density     4141
Name: density, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
y_train.shape

(10965, 1)

In [33]:
#Balanced Random Forest Classifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
rf_model = rf_model.fit(X_train, y_train)

In [34]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8041307020405606

In [35]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[ 842,  193],
       [ 538, 2083]])

In [36]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                    pre       rec       spe        f1       geo       iba       sup

high_density       0.61      0.81      0.79      0.70      0.80      0.65      1035
 low_density       0.92      0.79      0.81      0.85      0.80      0.65      2621

 avg / total       0.83      0.80      0.81      0.81      0.80      0.65      3656



In [37]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3691923467272658, 'num_people_qrt'),
 (0.3189255421171539, 'tot_people'),
 (0.03376831020198658, 'county_name_Los Angeles'),
 (0.024070757606416537, 'race_eth_code'),
 (0.02366851728626245, 'county_name_Riverside'),
 (0.01856709333690429, 'region_code'),
 (0.007893835656970834, 'county_name_San Bernardino'),
 (0.007080547864220715, 'region_name_Southern California'),
 (0.0069853235879066, 'county_name_Orange'),
 (0.0067137173808142015, 'race_eth_name_Latino'),
 (0.006537162680914342, 'race_eth_name_White'),
 (0.006345317728345141, 'race_eth_name_Asian'),
 (0.006285678941493319, 'race_eth_name_NHOPI'),
 (0.006112944787530083, 'county_name_Solano'),
 (0.00610848398482215, 'race_eth_name_AfricanAm'),
 (0.005718675768919875, 'region_name_Bay Area'),
 (0.005673570709838403, 'race_eth_name_Multiple'),
 (0.005402336738115066, 'race_eth_name_AIAN'),
 (0.004872520944902454, 'race_eth_name_Other'),
 (0.00476688567133717, 'race_eth_name_Total'),
 (0.004608627243264221, 'county_name_Madera'),


In [38]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1) 
ee_model = ee_model.fit(X_train, y_train)

In [39]:
# Calculated the balanced accuracy score
y_pred_ee = ee_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_ee)

0.872938381375254

In [40]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_ee)

array([[ 956,   79],
       [ 466, 2155]])

In [41]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ee))

                    pre       rec       spe        f1       geo       iba       sup

high_density       0.67      0.92      0.82      0.78      0.87      0.77      1035
 low_density       0.96      0.82      0.92      0.89      0.87      0.75      2621

 avg / total       0.88      0.85      0.89      0.86      0.87      0.76      3656

